# Attack viz

## Set up

In [2]:
import matplotlib
import pandas as pd
import pymysql
%matplotlib inline

In [3]:
connection = pymysql.connect(host='localhost',
                             user='fingerpatch',
                             password='fingerpatch',
                             db='fingerpatch',
                             charset='utf8mb4',
                             cursorclass=pymysql.cursors.DictCursor)

In [4]:
attack_table = pd.read_sql("SELECT * FROM `ubuntu_captures` ",connection)
ground_truth = pd.read_sql("SELECT * FROM `ubuntu_packets` ",connection)

In [11]:
ground_truth.count()[0]

128148

## Cleaning
Select interesting columns and remove duplicated rows

In [145]:
ground_truth.columns

Index(['capture_id', 'Package', 'Version', 'Architecture', 'Size',
       'Installed-Size', 'Priority', 'Maintainer', 'SHA1', 'Description',
       'parsedFrom', 'Description-md5', 'Bugs', 'Origin', 'MD5sum', 'Depends',
       'Homepage', 'Source', 'SHA256', 'Section', 'Supported', 'Filename',
       'packageMode', '#Depends', 'dist_from_expected_size'],
      dtype='object')

In [120]:
pMincluded = ground_truth.duplicated(['Package', 'Version', 'Size', 'Depends', 'packageMode']).sum()
pMexcluded = ground_truth.duplicated(['Package', 'Version', 'Size', 'Depends']).sum()
pMcat = ground_truth['packageMode'].unique()
print("By not considering packageMode which consists only in {}, we can withdraw: {} duplcates".format(pMcat ,pMexcluded - pMincluded))
ground_truth = ground_truth.drop_duplicates(['Package', 'Version', 'Size', 'Depends'])

By not considering packageMode which consists only in ['releases' 'source'], we can withdraw: 56997 duplcates


In [144]:
ground_truth.drop(axis= 1, columns=['capture_id','SHA1', 'Priority', 'Description-md5', 'MD5sum', 'SHA256', 'packageMode' ])

,Package,Version,Architecture,Size,Installed-Size,Maintainer,Description,parsedFrom,Bugs,Origin,Depends,Homepage,Source,Section,Supported,Filename,#Depends,dist_from_expected_size
0,apache2,2.4.10-1ubuntu1.1~ubuntu14.04.2,amd64,87626,464,Ubuntu Developers <ubuntu-devel-discuss@lists....,Apache HTTP Server,packages/archive.ubuntu.com_ubuntu_dists_trust...,https://bugs.launchpad.net/ubuntu/+filebug,Ubuntu,"lsb-base, procps, perl, mime-support, apache2-...",http://httpd.apache.org/,,web,5y,pool/main/a/apache2/apache2_2.4.10-1ubuntu1.1~...,6,20035
1,apache2-bin,2.4.10-1ubuntu1.1~ubuntu14.04.2,amd64,883356,3446,Ubuntu Developers <ubuntu-devel-discuss@lists....,Apache HTTP Server (binary files and modules),packages/archive.ubuntu.com_ubuntu_dists_trust...,https://bugs.launchpad.net/ubuntu/+filebug,Ubuntu,"libapr1 (>= 1.5.0), libaprutil1 (>= 1.5.0), li...",http://httpd.apache.org/,apache2,httpd,5y,pool/main/a/apache2/apache2-bin_2.4.10-1ubuntu...,12,815765
2,apache2-data,2.4.10-1ubuntu1.1~ubuntu14.04.2,all,159856,702,Ubuntu Developers <ubuntu-devel-discuss@lists....,Apache HTTP Server (common files),packages/archive.ubuntu.com_ubuntu_dists_trust...,https://bugs.launchpad.net/ubuntu/+filebug,Ubuntu,,http://httpd.apache.org/,apache2,httpd,5y,pool/main/a/apache2/apache2-data_2.4.10-1ubunt...,0,92265
3,apache2-dbg,2.4.10-1ubuntu1.1~ubuntu14.04.2,amd64,2044530,8627,Ubuntu Developers <ubuntu-devel-discuss@lists....,Apache debugging symbols,packages/archive.ubuntu.com_ubuntu_dists_trust...,https://bugs.launchpad.net/ubuntu/+filebug,Ubuntu,"apache2, apache2-bin (= 2.4.10-1ubuntu1.1~ubun...",http://httpd.apache.org/,apache2,debug,5y,pool/main/a/apache2/apache2-dbg_2.4.10-1ubuntu...,2,1976939
4,apache2-dev,2.4.10-1ubuntu1.1~ubuntu14.04.2,amd64,168318,930,Ubuntu Developers <ubuntu-devel-discuss@lists....,Apache HTTP Server (development headers),packages/archive.ubuntu.com_ubuntu_dists_trust...,https://bugs.launchpad.net/ubuntu/+filebug,Ubuntu,"openssl, libapr1-dev, libaprutil1-dev, debhelp...",http://httpd.apache.org/,apache2,httpd,5y,pool/main/a/apache2/apache2-dev_2.4.10-1ubuntu...,5,100727
5,apache2-doc,2.4.10-1ubuntu1.1~ubuntu14.04.2,all,2642890,19464,Ubuntu Developers <ubuntu-devel-discuss@lists....,Apache HTTP Server (on-site documentation),packages/archive.ubuntu.com_ubuntu_dists_trust...,https://bugs.launchpad.net/ubuntu/+filebug,Ubuntu,,http://httpd.apache.org/,apache2,doc,5y,pool/main/a/apache2/apache2-doc_2.4.10-1ubuntu...,0,2575299
6,apache2-mpm-event,2.4.10-1ubuntu1.1~ubuntu14.04.2,amd64,1468,22,Ubuntu Developers <ubuntu-devel-discuss@lists....,transitional event MPM package for apache2,packages/archive.ubuntu.com_ubuntu_dists_trust...,https://bugs.launchpad.net/ubuntu/+filebug,Ubuntu,apache2 (= 2.4.10-1ubuntu1.1~ubuntu14.04.2),http://httpd.apache.org/,apache2,web,5y,pool/main/a/apache2/apache2-mpm-event_2.4.10-1...,1,66123
7,apache2-mpm-prefork,2.4.10-1ubuntu1.1~ubuntu14.04.2,amd64,1470,22,Ubuntu Developers <ubuntu-devel-discuss@lists....,transitional prefork MPM package for apache2,packages/archive.ubuntu.com_ubuntu_dists_trust...,https://bugs.launchpad.net/ubuntu/+filebug,Ubuntu,apache2 (= 2.4.10-1ubuntu1.1~ubuntu14.04.2),http://httpd.apache.org/,apache2,net,5y,pool/main/a/apache2/apache2-mpm-prefork_2.4.10...,1,66121
8,apache2-mpm-worker,2.4.10-1ubuntu1.1~ubuntu14.04.2,amd64,1470,22,Ubuntu Developers <ubuntu-devel-discuss@lists....,transitional worker MPM package for apache2,packages/archive.ubuntu.com_ubuntu_dists_trust...,https://bugs.launchpad.net/ubuntu/+filebug,Ubuntu,apache2 (= 2.4.10-1ubuntu1.1~ubuntu14.04.2),http://httpd.apache.org/,apache2,net,5y,pool/main/a/apache2/apache2-mpm-worker_2.4.10-...,1,66121
9,apache2-utils,2.4.10-1ubuntu1.1~ubuntu14.04.2,amd64,83788,340,Ubuntu Developers <ubuntu-devel-discuss@lists....,Apache HTTP Server (utility programs for web s...,packages/archive.ubuntu.com_ubuntu_dists_trust...,https://bugs.launchpad.net/ubuntu/+filebug,Ubuntu,"libapr1 (>= 1.4.8-2~), libaprutil1 (>= 1.5.0),...",http:

## Mounting an Attack for matching a specific capture to a package.
##### Relying on package size

In [129]:
target = attack_table.loc[0]
target

capture_id                                     1
nb_flows                                       3
Flow1               target->danava.canonical.com
Flow2               danava.canonical.com->target
Flow3               target->danava.canonical.com
Flow4                                       None
Flow5                                       None
nb_Payload_send1                               0
nb_Payload_send2                           67874
nb_Payload_send3                             173
nb_Payload_send4                            None
nb_Payload_send5                            None
Name: 0, dtype: object

In [130]:
EXTRA_SIZE_AVERAGE = 283   # Made from stats about captured packets
EXTRA_SIZE_VARIATION = 5
size_to_match = target['nb_Payload_send2']

In [131]:
def distance_from_expected_average_size(x, size_to_match):
    return abs(size_to_match - x - EXTRA_SIZE_AVERAGE)

In [132]:
ground_truth["dist_from_expected_size"] = ground_truth["Size"].map(lambda x: distance_from_expected_average_size(x, size_to_match))

In [133]:
ground_truth.sort_values(by="dist_from_expected_size")

,capture_id,Package,Version,Architecture,Size,Installed-Size,Priority,Maintainer,SHA1,Description,...,Depends,Homepage,Source,SHA256,Section,Supported,Filename,packageMode,#Depends,dist_from_expected_size
39671,1,libopennebula-java-doc,3.4.1-4.1ubuntu1,all,67592,1194,extra,Ubuntu Developers <ubuntu-devel-discuss@lists....,b2449c56f7c0283ec320e83dbbf96164773b39b6,Java bindings for OpenNebula Cloud API (OCA) -...,...,,http://opennebula.org/,opennebula,61b560e07edf80a5fb3a3c59da5aed8e16b460b7c107cf...,universe/doc,,pool/universe/o/opennebula/libopennebula-java-...,releases,0,1
41518,1,libshisa-dev,1.0.2-3ubuntu2,amd64,67594,385,extra,Ubuntu Developers <ubuntu-devel-discuss@lists....,6cef545cfad241d0c4b1333e9396648a23094c56,Development files for the Shishi Kerberos v5 K...,...,"libshisa0 (= 1.0.2-3ubuntu2), libshishi-dev (=...",http://www.gnu.org/software/shishi/,shishi,7ea40bcc47247bca0449e641d92a9afea23851227bb580...,universe/libdevel,,pool/universe/s/shishi/libshisa-dev_1.0.2-3ubu...,releases,2,3
14153,1,libcloog-isl-dev,0.18.2-1,amd64,67588,377,optional,Ubuntu Developers <ubuntu-devel-discuss@lists....,ab29290b9279ffd0eabe34b9a7689fc5a3defd8e,Chunky Loop Generator (development files),...,"libisl-dev (>= 0.11), libgmp-dev, libcloog-isl...",http://www.CLooG.org,cloog,2a291cd9de63a5520da767ceaedbabad10ad88ae2ebc1a...,libdevel,9m,pool/main/c/cloog/libcloog-isl-dev_0.18.2-1_am...,releases,4,3
34921,1,libghc-shakespeare-i18n-prof,1.0.0.2-4build1,amd64,67582,429,extra,Ubuntu Developers <ubuntu-devel-discuss@lists....,157bebabbec9d14565590bdc4e6e7ef9a36bdbd5,type-based approach to internationalization; p...,...,libghc-shakespeare-i18n-dev (= 1.0.0.2-4build1...,http://hackage.haskell.org/package/shakespeare...,haskell-shakespeare-i18n,f1968377ff9f4c763d0cc5c1cc5f6d293624e3704cbc59...,universe/haskell,,pool/universe/h/haskell-shakespeare-i18n/libgh...,releases,8,9
26684,1,gkrellmoon,0.6-5,amd64,67578,320,optional,Ubuntu MOTU Developers <ubuntu-motu@lists.ubun...,49cd0f17854fb19287864d0d199521cd5abbd7e6,Gkrellm Moon Clock Plugin,...,"gkrellm (>= 2.0.0), libatk1.0-0 (>= 1.13.2), l...",,,43e853da209703fc5feab9d14ae18f311bf16efa57b04f...,universe/x11,,pool/universe/g/gkrellmoon/gkrellmoon_0.6-5_am...,releases,18,13
18783,1,python-troveclient,1:1.0.3-0ubuntu3,all,67612,473,extra,Ubuntu Developers <ubuntu-devel-discuss@lists....,5dda88b4bb8d3361082d7e3e7c07927f9dc2a465,Client library for Openstack Trove API.,...,"python-httplib2, python-lxml, python-setuptool...",,,28a77a6b3bdc9851d39a8eead4a54ce6d0edfa1a8d67e8...,python,5y,pool/main/p/python-troveclient/python-trovecli...,releases,12,21
9062,1,drizzle-plugin-regex-policy,1:7.2.3-2ubuntu2,amd64,67568,293,extra,Ubuntu Developers <ubuntu-devel-discuss@lists....,855a6149b939589cff251bb511723e1d50d543f9,Regex based authorization rules for Drizzle,...,"drizzle (= 1:7.2.3-2ubuntu2), libc6 (>= 2.14),...",http://launchpad.net/drizzle,drizzle,bb30184f0683a1ca3280033361774c9ac6107caeffad0c...,universe/database,,pool/universe/d/drizzle/drizzle-plugin-regex-p...,releases,4,23
1680,1,libapparmor-dev,2.10.95-0ubuntu2.6~14.04.3,amd64,67566,264,extra,Ubuntu Developers <ubuntu-devel-discuss@lists....,0e23e3831004dc8fd693a2f8327048953318fbe5,AppArmor development libraries and header files,...,libapparmor1 (= 2.10.95-0ubuntu2.6~14.04.3),http://apparmor.net/,apparmor,f6af27044aa35d85bd612c9e8d27fdb17ac2ebe622de16...,libdevel,5y,pool/main/a/apparmor/libapparmor-dev_2.10.95-0...,releases,1,25
52038,1,s3cmd,1.1.0~beta3-2,all,67562,288,optional,Ubuntu Developers <ubuntu-devel-discuss@lists....,bc4be512752d7fd34019fdf607a8d609b5cbd8e6,command-line Amazon S3 client,...,python (>= 2.7.1-0ubuntu2),http://s3tools.logix.cz/s3cmd,,a3710bf4f6b4ceace0d22e1578b7e56ab398b6aeab8fa0...,universe/admin,,pool/universe/s/s3cmd/s3cmd_1.1.0~beta3-2_all.deb,releases,1,29
10003,1,libkohana2-modules-php,2.3.4-2+deb7u1build0.14.04.1,all,67622,676,optional,Ubuntu Developers <ubuntu-devel-discuss@lists....,eebfb9c17c8346edd38b04f24df0da2641

The first one is the one.

In [134]:
ground_truth.sort_values(by="dist_from_expected_size").iloc[0]

capture_id                                                                 1
Package                                               libopennebula-java-doc
Version                                                     3.4.1-4.1ubuntu1
Architecture                                                             all
Size                                                                   67592
Installed-Size                                                          1194
Priority                                                               extra
Maintainer                 Ubuntu Developers <ubuntu-devel-discuss@lists....
SHA1                                b2449c56f7c0283ec320e83dbbf96164773b39b6
Description                Java bindings for OpenNebula Cloud API (OCA) -...
parsedFrom                 packages/archive.ubuntu.com_ubuntu_dists_trust...
Description-md5                             bd5164b179b060e59f8e811d417ecd61
Bugs                              https://bugs.launchpad.net/ubuntu/+filebug

## Dependens discovery

let's take only the packages that have only one depends.

Sort them by ascending size

In [135]:
tmp = ground_truth
tmp["#Depends"] = tmp["Depends"].map(lambda x: 0 if x == "" else len(x.split(",")))
two_dep_first10 = tmp[tmp["#Depends"] == 1].sort_values(by = "Size", ascending=True)[:10]

In [136]:
two_dep_first10.iloc[0]

capture_id                                                                 1
Package                                                              readpst
Version                                                       0.6.59-1build1
Architecture                                                             all
Size                                                                     796
Installed-Size                                                            21
Priority                                                            optional
Maintainer                 Ubuntu Developers <ubuntu-devel-discuss@lists....
SHA1                                da3b4b8e1da9e72c3f8a82d03fd620a8590c7186
Description                    Converts Outlook PST files to mbox and others
parsedFrom                 packages/archive.ubuntu.com_ubuntu_dists_trust...
Description-md5                             3ef2b339ca5bf4fb7de6354b65f7cf59
Bugs                              https://bugs.launchpad.net/ubuntu/+filebug

Seeking about that depends

In [137]:
ground_truth[ground_truth["Package"] == two_dep_first10.iloc[0]["Depends"]]

,capture_id,Package,Version,Architecture,Size,Installed-Size,Priority,Maintainer,SHA1,Description,...,Depends,Homepage,Source,SHA256,Section,Supported,Filename,packageMode,#Depends,dist_from_expected_size
18279,1,pst-utils,0.6.59-1build1,amd64,62092,181,optional,Ubuntu Developers <ubuntu-devel-discuss@lists....,7e5afc4156a87f8c02ae369428ffcbb967111bed,tools for reading Microsoft Outlook PST files,...,"libc6 (>= 2.14), libgcc1 (>= 1:4.1.1), libgd3 ...",http://www.five-ten-sg.com/libpst/,libpst,807a10cc7b8062403018d37a6d19cd210bd9f836ae62d9...,utils,9m,pool/main/libp/libpst/pst-utils_0.6.59-1build1...,releases,7,5499


Turns out that this Dependens also have its Dependances

In [138]:
sub_dependances = ground_truth[ground_truth["Package"] == two_dep_first10.iloc[0]["Depends"]].iloc[0]["Depends"]
print(sub_dependances)

libc6 (>= 2.14), libgcc1 (>= 1:4.1.1), libgd3 (>= 2.1.0~alpha~), libglib2.0-0 (>= 2.12.0), libgsf-1-114 (>= 1.14.8), libpst4 (>= 0.6.54), libstdc++6 (>= 4.6)


#### Once Downloading the package we can ideed see that the package doesn't just depends on one single package but many subpackages from that dependance:

On the Docker:

`The following extra packages will be installed:
  fontconfig-config fonts-dejavu-core libfontconfig1 libfreetype6 libgd3
  libglib2.0-0 libglib2.0-data libgsf-1-114 libgsf-1-common libjbig0
  libjpeg-turbo8 libjpeg8 libpst4 libtiff5 libvpx1 libx11-6 libx11-data
  libxau6 libxcb1 libxdmcp6 libxml2 libxpm4 pst-utils sgml-base
  shared-mime-info xml-core`
  
`0 upgraded, 27 newly installed, 0 to remove and 32 not upgraded.
Need to get 5664 kB of archives.`


On the attacker:

`historic =  ['target->danava.canonical.com', 'danava.canonical.com->target', 'target->danava.canonical.com']
server_ip =  ['91.189.88.149', '172.100.0.100', '91.189.88.149']
server_name =  ['danava.canonical.com', 'target', 'danava.canonical.com']
received_Payload =  [5671834]
send_Payload =  [0, 4251]`

So if we calculate with the tipical extra_size for each downloaded package that we get on the attacker side and knowing that 5664kB is rounded:


In [139]:
EXTRA_SIZE_AVERAGE * 27 + 5664000

5671641

In [146]:
connection.close()